# HW from CPC


CPC Global Unified Temperature data products from PSL

Citation Please note: If you acquire CPC Global Unified Temperature data products from PSL, we ask that you acknowledge us in your use 
of the data. This may be done by including text such as CPC Global Unified Temperature data provided by the NOAA PSL, Boulder, Colorado, USA,
from their website at https://psl.noaa.gov in any documents or publications using these data. We would also appreciate receiving a copy of 
the relevant publications. This will help PSL to justify keeping the data freely available online in the future. Thank you!

Opendap
download: https://psl.noaa.gov/thredds/catalog/Datasets/cpc_global_temp/catalog.html?dataset=Datasets/cpc_global_temp/


In [2]:
import netCDF4
import time
from netCDF4 import Dataset
import geopandas as gpd
import numpy as np
%pylab inline
import netCDF4
from netCDF4 import Dataset
from datetime import datetime
from scipy import signal
#from pylab import *
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import *
from matplotlib.cm import ScalarMappable
import time


from xarray.backends import NetCDF4DataStore
import xarray as xr
import xarray as xr
import geopandas as gpd



%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
def download_CPC_opendap(start,end):
    """downloading CPC from opendap"""
    time_range = np.arange(start,end+1)
    tmax = {}
    if start <1979:
        print('Start should be >=1979')
    else:
        for e,u in enumerate(time_range):
            print(e,u)
            url = 'http://psl.noaa.gov/thredds/dodsC/Datasets/cpc_global_temp/tmax.%s.nc'%u
            nc = Dataset(url).variables
            lon = nc['lon'][:]
            lat = nc['lat'][:]
            # BRAZIL section
            lon_min= np.abs(lon[:] -(360-75)).argmin()
            lon_max= np.abs(lon[:] -(360-34)).argmin()
            
            lat_min= np.abs(lat[:] -(6)).argmin()
            lat_max= np.abs(lat[:] -(-35)).argmin()
            tmax[e] = ma.masked_values(nc['tmax'][:,lat_min:lat_max,lon_min:lon_max],nc['tmax'].missing_value)
            #print(tmax.shape)
            #time, lats,lons = tmax.shape
            #HW = {} #ma.masked_all((lats,lons))
            
            #for i in range (lats):
            #    for j in range(lons):
            #        HW = HEATWAVE_SUM_YEAR(tmax[:,i,j],tmaxclim[i,j])

    


    return tmax
CPC_tmax = download_CPC_opendap(2011,2023)
CPC_clim = download_CPC_opendap(1981,2010)

import pickle
path = '/Users/calim/code/CEMADEN/data/'
pickle.dump(CPC_clim, open( path+"CPC_1981_2010.p", "wb" ) )
pickle.dump(CPC_tmax, open( path+"CPC_tmax.p", "wb" ) )

In [ ]:
confind = 95 # 95% above 3 consecutives days

In [ ]:
#i,j=0,0
#time, lats,lons = tmax_2011.shape
#heatwave={}
#for i in range (lats):
#    for j in range(lons):
def HEATWAVE_SUM_YEAR(tmax,tmax_clim):
    """Heat wave 
       periods of consecutive days with Tmax values above a certain percentile of 
       Tmax for the particular calendar day (calculated on a 15 day window). 
       Different percentiles (80th, 90th, 95th) 
       return:
       np.mean(HW) = mean temp of a HW
       np.sum(htwv_amp) = total days under HW
       len(htwv_amp) = frequency of HW
       np.mean(htwv_amp) = duration of HW
    """
    #heatwave[i,j] = np.where(tmax[:,i,j]>tmax_clim[i,j])[0]
    heatwave = np.where(tmax>tmax_clim)[0]
    #print(heatwave[i,j])
    prev=np.zeros(len(heatwave)+1)
    low=np.zeros(len(heatwave))
    count=0 
    for a in range (1,len(heatwave)):
        if heatwave[a] == heatwave[a-1]+1:  #loop each index above confind%
            count+=1                              #if days consecutive add 1
            prev[a]=count 
        else:
            count =0                              #if day is not consecutive =0
    np.where(prev==0)[0]
    low = []
    low = np.where(prev==0)[0] #SEARCH FOR EMPTY SPACES
    htwv_amp,HW=[],[]
    days_hw =[]
    for b in range (1,len(low)):
        above90 = heatwave[low[b-1]:low[b]] 
        if b == len(low):
            above90 =(heatwave[low[b-1]:len(low)])
        if (len(above90)>=3):
            htwv_amp.append(len(above90)) # HW amplitude
            days_hw.append(above90[0])

    return np.sum(htwv_amp),len(htwv_amp),(ma.mean(htwv_amp))
            



In [ ]:
import pickle
path = '/user/data/'
#CPC_clim = pickle.load(open(path+"CPC_1981_2010.p", 'rb'))
CPC_tmax = pickle.load(open(path+"CPC_tmax.p", 'rb'))
#CPC_clim_y = pickle.load(open(path+"CPC_clim_y", 'rb'))
CPC_clim_y_90 = pickle.load(open(path+"CPC_clim_y_90.p", 'rb'))

time, lats,lons = CPC_tmax[0].shape
HW, freqHW,durHW = ma.masked_all((len(CPC_tmax),lats,lons)),ma.masked_all((len(CPC_tmax),lats,lons)),ma.masked_all((len(CPC_tmax),lats,lons))
for e in range (len(CPC_tmax)):

    for i in range (lats):
        for j in range(lons):
            HW[e,i,j],freqHW[e,i,j],durHW[e,i,j] = HEATWAVE_SUM_YEAR(CPC_tmax[e][:,i,j],tmax_clim_annual[i,j])

import pickle
path = '/user/data/'
pickle.dump(HW, open( path+"HW.p", "wb" ) )
pickle.dump(freqHW, open( path+"freqHW.p", "wb" ) )
pickle.dump(durHW, open( path+"durHW.p", "wb" ) )